# Data Mining Project 3

### Datasets used: 
### 1. Odi-mens-cricket-match-data-2002-2023
### 2. icc-mens-world-cup-2023 data

In [273]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score 

In [274]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [275]:
from tensorflow import keras
from keras.utils import to_categorical

In [276]:
pd.set_option('display.max_columns', None)

# Task 3

**Loading Datasets**

In [277]:
df = pd.read_csv('ODI_Match_info.csv')

In [278]:
df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1389389,2023/24,Indore,2023/09/24,India,Australia,Australia,field,D/L,1,India,99,0,SS Iyer,"Holkar Cricket Stadium, Indore",J Madanagopal,HDPK Dharmasena,KN Ananthapadmanabhan
1,1336129,2023,Nottingham,2023/09/23,England,Ireland,Ireland,field,normal,0,England,48,0,WG Jacks,"Trent Bridge, Nottingham",DJ Millns,RJ Tucker,PR Reiffel
2,1395701,2023,Dhaka,2023/09/23,New Zealand,Bangladesh,New Zealand,bat,normal,0,New Zealand,86,0,IS Sodhi,"Shere Bangla National Stadium, Mirpur",M Erasmus,Sharfuddoula,Nitin Menon
3,1389388,2023/24,Chandigarh,2023/09/22,Australia,India,India,field,normal,0,India,0,5,Mohammed Shami,"Punjab Cricket Association IS Bindra Stadium, ...",KN Ananthapadmanabhan,HDPK Dharmasena,J Madanagopal
4,1395700,2023,Dhaka,2023/09/21,New Zealand,Bangladesh,Bangladesh,field,normal,0,NaN,0,0,NaN,"Shere Bangla National Stadium, Mirpur",Nitin Menon,Sharfuddoula,M Erasmus


In [279]:
df3 = pd.read_csv('matches.csv')

In [280]:
df3.head()

,season,team1,team2,date,match_number,venue,city,toss_winner,toss_decision,player_of_match,umpire1,umpire2,reserve_umpire,match_referee,winner,winner_runs,winner_wickets,match_type
0,2023/24,England,New Zealand,2023/10/05,1,Narendra Modi Stadium,Ahmedabad,New Zealand,field,R Ravindra,HDPK Dharmasena,Nitin Menon,Sharfuddoula,J Srinath,New Zealand,NaN,9.0,Group
1,2023/24,Pakistan,Netherlands,2023/10/06,2,Rajiv Gandhi International Stadium,Hyderabad,Netherlands,field,Saud Shakeel,AT Holdstock,CM Brown,PR Reiffel,JJ Crowe,Pakistan,81.0,NaN,Group
2,2023/24,Afghanistan,Bangladesh,2023/10/07,3,Himachal Pradesh Cricket Association Stadium,Dharamsala,Bangladesh,field,Mehedi Hasan Miraz,JS Wilson,HDPK Dharmasena,M Erasmus,AJ Pycroft,Bangladesh,NaN,6.0,Group
3,2023/24,South Africa,Sri Lanka,2023/10/07,4,Arun Jaitley Stadium,Delhi,Sri Lanka,field,AK Markram,RK Illingworth,Sharfuddoula,AG Wharf,J Srinath,South Africa,102.0,NaN,Group
4,2023/24,Australia,India,2023/10/08,5,MA Chidambaram Stadium,Chennai,Australia,bat,KL Rahul,CB Gaffaney,RA Kettleborough,AT Holdstock,RB Richardson,India,NaN,6.0,Group


In [281]:
df3.shape

(32, 18)

**listing columns**

In [282]:
df.columns.to_list(), len(df.columns)

(['id',
  'season',
  'city',
  'date',
  'team1',
  'team2',
  'toss_winner',
  'toss_decision',
  'result',
  'dl_applied',
  'winner',
  'win_by_runs',
  'win_by_wickets',
  'player_of_match',
  'venue',
  'umpire1',
  'umpire2',
  'umpire3'],
 18)

In [283]:
df3.columns.to_list(), len(df3.columns)

(['season',
  'team1',
  'team2',
  'date',
  'match_number',
  'venue',
  'city',
  'toss_winner',
  'toss_decision',
  'player_of_match',
  'umpire1',
  'umpire2',
  'reserve_umpire',
  'match_referee',
  'winner',
  'winner_runs',
  'winner_wickets',
  'match_type'],
 18)

Dropping columns that contain data which is obtained after match

In [284]:
df3 = df3.drop(['match_type','winner_runs','winner_wickets','match_referee'], axis = 1)
df = df.drop(['dl_applied','win_by_runs','win_by_wickets','result'], axis = 1)

In [285]:
df.columns.to_list(),df3.columns.to_list(),len(df.columns),len(df3.columns)

(['id',
  'season',
  'city',
  'date',
  'team1',
  'team2',
  'toss_winner',
  'toss_decision',
  'winner',
  'player_of_match',
  'venue',
  'umpire1',
  'umpire2',
  'umpire3'],
 ['season',
  'team1',
  'team2',
  'date',
  'match_number',
  'venue',
  'city',
  'toss_winner',
  'toss_decision',
  'player_of_match',
  'umpire1',
  'umpire2',
  'reserve_umpire',
  'winner'],
 14,
 14)

On listing columns again we see that majority of columns have the same meaning but have different names

In [286]:
df3.head(2)

,season,team1,team2,date,match_number,venue,city,toss_winner,toss_decision,player_of_match,umpire1,umpire2,reserve_umpire,winner
0,2023/24,England,New Zealand,2023/10/05,1,Narendra Modi Stadium,Ahmedabad,New Zealand,field,R Ravindra,HDPK Dharmasena,Nitin Menon,Sharfuddoula,New Zealand
1,2023/24,Pakistan,Netherlands,2023/10/06,2,Rajiv Gandhi International Stadium,Hyderabad,Netherlands,field,Saud Shakeel,AT Holdstock,CM Brown,PR Reiffel,Pakistan


renaming columns in order to append dataframes

In [287]:
df3 = df3.rename(columns={"match_number":"id","reserve_umpire":"umpire3"})
df3.columns.to_list()

['season',
 'team1',
 'team2',
 'date',
 'id',
 'venue',
 'city',
 'toss_winner',
 'toss_decision',
 'player_of_match',
 'umpire1',
 'umpire2',
 'umpire3',
 'winner']

reordering columns in order to append dataframes

In [288]:
df3 = df3[df.columns]

Checking the id column of both dataframes

In [289]:
df[df['season']=="2023/24"].head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,umpire3
0,1389389,2023/24,Indore,2023/09/24,India,Australia,Australia,field,India,SS Iyer,"Holkar Cricket Stadium, Indore",J Madanagopal,HDPK Dharmasena,KN Ananthapadmanabhan
3,1389388,2023/24,Chandigarh,2023/09/22,Australia,India,India,field,India,Mohammed Shami,"Punjab Cricket Association IS Bindra Stadium, ...",KN Ananthapadmanabhan,HDPK Dharmasena,J Madanagopal
6,1373575,2023/24,Johannesburg,2023/09/17,South Africa,Australia,Australia,field,South Africa,M Jansen,"The Wanderers Stadium, Johannesburg",A Paleker,RA Kettleborough,Nitin Menon
9,1373574,2023/24,Centurion,2023/09/15,South Africa,Australia,Australia,field,South Africa,H Klaasen,"SuperSport Park, Centurion",A Paleker,Nitin Menon,RA Kettleborough
12,1373573,2023/24,Potchefstroom,2023/09/12,South Africa,Australia,Australia,field,South Africa,AK Markram,"Sedgars Park, Potchefstroom",BP Jele,RA Kettleborough,Nitin Menon


In [290]:
df3[df3['season']=="2023/24"].head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2023/24,Ahmedabad,2023/10/05,England,New Zealand,New Zealand,field,New Zealand,R Ravindra,Narendra Modi Stadium,HDPK Dharmasena,Nitin Menon,Sharfuddoula
1,2,2023/24,Hyderabad,2023/10/06,Pakistan,Netherlands,Netherlands,field,Pakistan,Saud Shakeel,Rajiv Gandhi International Stadium,AT Holdstock,CM Brown,PR Reiffel
2,3,2023/24,Dharamsala,2023/10/07,Afghanistan,Bangladesh,Bangladesh,field,Bangladesh,Mehedi Hasan Miraz,Himachal Pradesh Cricket Association Stadium,JS Wilson,HDPK Dharmasena,M Erasmus
3,4,2023/24,Delhi,2023/10/07,South Africa,Sri Lanka,Sri Lanka,field,South Africa,AK Markram,Arun Jaitley Stadium,RK Illingworth,Sharfuddoula,AG Wharf
4,5,2023/24,Chennai,2023/10/08,Australia,India,Australia,bat,India,KL Rahul,MA Chidambaram Stadium,CB Gaffaney,RA Kettleborough,AT Holdstock


Changing the id of 2023 match dataframe , all matches of 2023 world cup will have same id , this id will be equal to one plus max id value of 2002-2023 dataframe 

In [291]:
df3['id']=df['id'].max()+1

**Concatenating present world cup data and previous odi data**

In [292]:
df = pd.concat([df, df3], sort=False)

## Handling Null Values and dropping columns

In [293]:
df.isna().sum()

id                   0
season               0
city               310
date                 0
team1                0
team2                0
toss_winner          0
toss_decision        0
winner             120
player_of_match    151
venue                0
umpire1              0
umpire2              0
umpire3            283
dtype: int64

**removing matches where winner not concluded**

In [294]:
df = df.dropna(axis = 0,subset=['winner'])

In [295]:
df[df['player_of_match'].isna()].head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,umpire3
165,1341974,2022/23,Windhoek,2022/11/26,United States of America,Namibia,Namibia,field,Namibia,NaN,"Wanderers Cricket Ground, Windhoek",A Louw,BP Jele,NaN
183,1331369,2022,Port Moresby,2022/09/20,Namibia,Papua New Guinea,Namibia,bat,Namibia,NaN,"Amini Park, Port Moresby",DM Koch,L Oala,NaN
185,1331367,2022,Port Moresby,2022/09/15,Namibia,United States of America,Namibia,bat,Namibia,NaN,"Amini Park, Port Moresby",A Kapa,L Oala,NaN
186,1331366,2022,Port Moresby,2022/09/13,Papua New Guinea,United States of America,United States of America,field,Papua New Guinea,NaN,"Amini Park, Port Moresby",A Kapa,DM Koch,NaN
254,1312802,2022,Pearland,2022/06/03,Scotland,United Arab Emirates,Scotland,bat,United Arab Emirates,NaN,"Moosa Cricket Stadium, Pearland",SN Bandekar,Vijaya Mallela,NaN


Removing all columns containing only null values

In [296]:
df = df.dropna(how='all')

In [297]:
df.isna().sum()

id                   0
season               0
city               297
date                 0
team1                0
team2                0
toss_winner          0
toss_decision        0
winner               0
player_of_match     57
venue                0
umpire1              0
umpire2              0
umpire3            268
dtype: int64

Removing umpire2 and umpire3 column

In [298]:
df = df.drop(['umpire2','umpire3'], axis = 1)

In [299]:
df[df['city'].isna()].head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1
383,1223956,2020/21,NaN,2020/11/29,Australia,India,Australia,bat,Australia,SPD Smith,Sydney Cricket Ground,GA Abood
384,1223955,2020/21,NaN,2020/11/27,Australia,India,Australia,bat,Australia,SPD Smith,Sydney Cricket Ground,RJ Tucker
386,1233462,2020/21,NaN,2020/11/01,Zimbabwe,Pakistan,Zimbabwe,bat,Pakistan,Iftikhar Ahmed,Rawalpindi Cricket Stadium,Asif Yaqoob
387,1233461,2020/21,NaN,2020/10/30,Pakistan,Zimbabwe,Pakistan,bat,Pakistan,BRM Taylor,Rawalpindi Cricket Stadium,Aleem Dar
579,1144987,2018/19,NaN,2018/11/09,Australia,South Africa,South Africa,field,Australia,AJ Finch,Adelaide Oval,MA Gough


filling in null city column values using venue column, 
for example some other row may contain city column value where venue is "Sydney Cricket Stadium", 
hence filling these value 

In [300]:
# First, create a dictionary mapping venues to their corresponding cities
venue_city_mapping = df.dropna(subset=['city']).set_index('venue')['city'].to_dict()

# Function to fill missing city values based on venue
def fill_missing_city(row):
    if pd.isnull(row['city']):
        return venue_city_mapping.get(row['venue'], row['city'])
    return row['city']

# Apply the function to fill missing city values
df['city'] = df.apply(fill_missing_city, axis=1)


In [301]:
 df[df['city'].isna()]


,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1
581,1144986,2018/19,NaN,2018/11/04,Australia,South Africa,South Africa,field,South Africa,DW Steyn,Perth Stadium,Aleem Dar
675,1072314,2017/18,NaN,2018/01/28,England,Australia,Australia,field,England,TK Curran,Perth Stadium,CB Gaffaney
739,1104479,2017,NaN,2017/07/02,Sri Lanka,Zimbabwe,Sri Lanka,field,Sri Lanka,PADLR Sandakan,Galle International Stadium,NJ Llong
741,1104478,2017,NaN,2017/06/30,Sri Lanka,Zimbabwe,Sri Lanka,bat,Zimbabwe,SF Mire,Galle International Stadium,IJ Gould
1170,661689,2013/14,NaN,2014/01/01,New Zealand,West Indies,West Indies,field,New Zealand,CJ Anderson,Queenstown Events Centre,CB Gaffaney
1362,543882,2011/12,NaN,2012/02/20,Kenya,Ireland,Ireland,bat,Ireland,EC Joyce,Mombasa Sports Club Ground,S George
1365,543881,2011/12,NaN,2012/02/18,Kenya,Ireland,Ireland,bat,Kenya,T Mishra,Mombasa Sports Club Ground,D Odhiambo
1701,392617,2009,NaN,2009/04/24,Australia,Pakistan,Pakistan,bat,Australia,A Symonds,Dubai Sports City Cricket Stadium,Asad Rauf
1702,392616,2009,NaN,2009/04/22,Australia,Pakistan,Australia,bat,Pakistan,Shahid Afridi,Dubai Sports City Cricket Stadium,BF Bowden
1814,323949,2007/08,NaN,2008/03/09,Bangladesh,South Africa,Bangladesh,bat,South Africa,GC Smith,Chittagong Divisional Stadium,Aleem Dar


In [302]:
df[df['venue']== 'Perth Stadium']

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1
581,1144986,2018/19,NaN,2018/11/04,Australia,South Africa,South Africa,field,South Africa,DW Steyn,Perth Stadium,Aleem Dar
675,1072314,2017/18,NaN,2018/01/28,England,Australia,Australia,field,England,TK Curran,Perth Stadium,CB Gaffaney


filling null city values left out by above method

In [303]:
df.loc[df['venue'] == 'Perth Stadium', 'city'] = 'Australia'
df.loc[df['venue'] == 'Galle International Stadium', 'city'] = 'Sri Lanka'
df.loc[df['venue'] == 'Queenstown Events Centre', 'city'] = 'New Zealand'
df.loc[df['venue'] == 'Mombasa Sports Club Ground', 'city'] = 'Kenya'
df.loc[df['venue'] == 'Dubai Sports City Cricket Stadium', 'city'] = 'United Arab Emirates'
df.loc[df['venue'] == 'Chittagong Divisional Stadium', 'city'] = 'Bangladesh'
df.loc[df['venue'] == 'Sheikhupura Stadium', 'city'] = 'Pakistan'

In [304]:
df.isna().sum()

id                  0
season              0
city                0
date                0
team1               0
team2               0
toss_winner         0
toss_decision       0
winner              0
player_of_match    57
venue               0
umpire1             0
dtype: int64

converting date to dayofyear

In [305]:
df['day_of_year'] = pd.to_datetime(df['date'], format='%Y/%m/%d')

# Extract the day of the year and create a new column
df['day_of_year'] = df['day_of_year'].dt.dayofyear

In [306]:
df = df.drop(['date'], axis = 1)

In [307]:
df.head()

,id,season,city,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,day_of_year
0,1389389,2023/24,Indore,India,Australia,Australia,field,India,SS Iyer,"Holkar Cricket Stadium, Indore",J Madanagopal,267
1,1336129,2023,Nottingham,England,Ireland,Ireland,field,England,WG Jacks,"Trent Bridge, Nottingham",DJ Millns,266
2,1395701,2023,Dhaka,New Zealand,Bangladesh,New Zealand,bat,New Zealand,IS Sodhi,"Shere Bangla National Stadium, Mirpur",M Erasmus,266
3,1389388,2023/24,Chandigarh,Australia,India,India,field,India,Mohammed Shami,"Punjab Cricket Association IS Bindra Stadium, ...",KN Ananthapadmanabhan,265
5,1388414,2023,Colombo,Sri Lanka,India,Sri Lanka,bat,India,Mohammed Siraj,"R Premadasa Stadium, Colombo",Ahmed Shah Pakteen,260


venue has both values 'Wankhede Stadium' and 'Wankhede Stadium, Mumbai' , removing location from venue

In [308]:
df['venue'] = df['venue'].str.split(',').str.get(0)

In [309]:
new_df = df[['season', 'city', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'venue', 'umpire1', 
       'day_of_year']]

In [310]:
df['winner'].value_counts()

India                       296
Australia                   279
South Africa                218
Sri Lanka                   216
England                     210
Pakistan                    202
New Zealand                 188
West Indies                 136
Bangladesh                  129
Zimbabwe                     67
Afghanistan                  59
Ireland                      53
Scotland                     45
United Arab Emirates         34
Nepal                        28
Oman                         22
United States of America     22
Namibia                      21
Netherlands                  19
Kenya                        13
Papua New Guinea             11
Canada                       10
Hong Kong                     6
Asia XI                       3
Bermuda                       2
Jersey                        1
Africa XI                     1
Name: winner, dtype: int64

removing some teams with less number of matches won

In [311]:
values_to_drop = ['Scotland',  
'United Arab Emirates',      
'Nepal',                        
'Oman',                         
'United States of America',    
'Namibia',                    
'Netherlands',                  
'Kenya',                        
'Papua New Guinea',             
'Canada',                       
'Hong Kong',                    
'Asia XI',                    
'Bermuda',                    
'Jersey',                        
'Africa XI',
'ICC World XI' ]

# Drop rows where the 'winner' column has values in 'values_to_drop'
new_df = df[~df['winner'].isin(values_to_drop)]
new_df = new_df[~new_df['team1'].isin(values_to_drop)]
new_df = new_df[~new_df['team2'].isin(values_to_drop)]

In [312]:
new_df['team2'].value_counts(), new_df.shape

(Sri Lanka       249
 Pakistan        234
 India           223
 Australia       180
 West Indies     175
 South Africa    171
 England         169
 New Zealand     154
 Zimbabwe        124
 Bangladesh      111
 Afghanistan      51
 Ireland          50
 Name: team2, dtype: int64,
 (1891, 12))

## Encoding Data

In [313]:
TeamMapping ={'India':1, 'Australia':2,'South Africa':3,'Sri Lanka':4,'England':5,'Pakistan':6,'New Zealand':7,     
'West Indies':8,'Bangladesh':9,'Zimbabwe':10,'Afghanistan':11,'Ireland':12}
new_df['team1'] = new_df['team1'].map(TeamMapping)

new_df['team2'] = new_df['team2'].map(TeamMapping)

new_df['winner'] = new_df['winner'].map(TeamMapping)

new_df['toss_winner'] = new_df['toss_winner'].map(TeamMapping)

In [314]:
new_df.isna().sum()

id                  0
season              0
city                0
team1               0
team2               0
toss_winner         0
toss_decision       0
winner              0
player_of_match    18
venue               0
umpire1             0
day_of_year         0
dtype: int64

In [315]:
#creating a copyof dataframe
copy_df = new_df.copy()

In [316]:
le = LabelEncoder()

new_df['season'] = le.fit_transform(new_df['season'])
# Create a dictionary mapping original values to encoded values
mapping_season = dict(zip(copy_df['season'], new_df['season']))

new_df['toss_decision'] = le.fit_transform(new_df['toss_decision'])
# Create a dictionary mapping original values to encoded values
mapping_toss_decision = dict(zip(copy_df['toss_decision'], new_df['toss_decision']))

new_df['city'] = le.fit_transform(new_df['city'])
# Create a dictionary mapping original values to encoded values
mapping_city = dict(zip(copy_df['city'], new_df['city']))

new_df['venue'] = le.fit_transform(new_df['venue'])
# Create a dictionary mapping original values to encoded values
mapping_venue = dict(zip(copy_df['venue'], new_df['venue']))

new_df['umpire1'] = le.fit_transform(new_df['umpire1'])
# Create a dictionary mapping original values to encoded values
mapping_umpire = dict(zip(copy_df['umpire1'], new_df['umpire1']))

In [317]:
new_df.tail()

,id,season,city,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,day_of_year
26,1395702,42,41,2,7,7,1,2,TM Head,55,62,301
28,1395702,42,82,1,5,5,1,1,RG Sharma,14,7,302
29,1395702,42,105,4,11,11,1,11,Fazalhaq Farooqi,76,25,303
30,1395702,42,77,9,6,9,0,6,Fakhar Zaman,41,76,304
31,1395702,42,105,3,7,7,1,3,HE van der Dussen,76,12,305


In [318]:
new_df.shape

(1891, 12)

## Training model to predict winner of match

y which is the winner column is the target variable

In [319]:
X = new_df[['season', 'city', 'team1', 'team2', 'toss_winner', 'toss_decision',
        'venue', 'umpire1', 'day_of_year']]
y = new_df['winner']

## knn

n_neighbors hyperparameter taken as 6, other hyperparameters taken as default values of library

In [320]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [321]:
neigh = KNeighborsClassifier(n_neighbors=6)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=6)

In [322]:
y_pred = neigh.predict(X_test)

In [323]:
print(f"Accuracy:{accuracy_score(y_test, y_pred)}")
print(f"Precision:{precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall:{recall_score(y_test, y_pred, average='weighted')}")
print(f"f1 score:{f1_score(y_test, y_pred, average='weighted')}")

Accuracy:0.2638522427440633
Precision:0.26336782107401824
Recall:0.2638522427440633
f1 score:0.2590502752734971


Applied k fold cross validation to confirm performance metrics

In [324]:
k_fold = KFold(n_splits = 5, random_state = None) 
score = cross_val_score(neigh, X, y, cv = k_fold)  
  
# Calculating mean accuracy score  
mean_acc_score = sum(score) / len(score)  
   
print("Accuracy score of each fold: ", score)  
print("Mean accuracy score: ", mean_acc_score)  

Accuracy score of each fold:  [0.18469657 0.21428571 0.20899471 0.26190476 0.2010582 ]
Mean accuracy score:  0.2141879912328461


## naive bayes

In [325]:
bayes = CategoricalNB()
bayes.fit(X_train, y_train)


CategoricalNB()

In [326]:
y_pred = bayes.predict(X_test)

In [327]:
print(f"Accuracy:{accuracy_score(y_test, y_pred)}")
print(f"Precision:{precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall:{recall_score(y_test, y_pred, average='weighted')}")
print(f"f1 score:{f1_score(y_test, y_pred, average='weighted')}")

Accuracy:0.5725593667546174
Precision:0.5948567934868707
Recall:0.5725593667546174
f1 score:0.5761990461266114


## random forest

max_depth hyperparameter taken as 10, other hyperparameters taken as default values of library

In [328]:
forest = RandomForestClassifier(max_depth=10, random_state=0)
forest.fit(X_train, y_train)


y_pred = forest.predict(X_test)

In [329]:
print(f"Accuracy:{accuracy_score(y_test, y_pred)}")
print(f"Precision:{precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall:{recall_score(y_test, y_pred, average='weighted')}")
print(f"f1 score:{f1_score(y_test, y_pred, average='weighted')}")

Accuracy:0.6200527704485488
Precision:0.6382601914871178
Recall:0.6200527704485488
f1 score:0.6077403447367002


Applied k fold cross validation to confirm performance metrics

In [330]:
k_fold = KFold(n_splits = 5, random_state = None) 
score = cross_val_score(forest, X, y, cv = k_fold)  
  
# Calculating mean accuracy score  
mean_acc_score = sum(score) / len(score)  
   
print("Accuracy score of each fold: ", score)  
print("Mean accuracy score: ", mean_acc_score)  

Accuracy score of each fold:  [0.65435356 0.57671958 0.61375661 0.65873016 0.61375661]
Mean accuracy score:  0.623463304993648


## Using Neural Network

The neural network used here is multi layer perceptron provided by sklearn
Contains 4 hidden layers 

In [331]:
mlp = MLPClassifier(hidden_layer_sizes=(22,100,100,22),random_state=1, max_iter=300).fit(X_train, y_train)
y_pred = mlp.predict(X_test)

C:\Users\dhair\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [332]:
y_pred2 = mlp.predict(X_train)
accuracy_score(y_train, y_pred2)

0.5866402116402116

In [333]:
print(f"Accuracy:{accuracy_score(y_test, y_pred)}")
print(f"Precision:{precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall:{recall_score(y_test, y_pred, average='weighted')}")
print(f"f1 score:{f1_score(y_test, y_pred, average='weighted')}")

Accuracy:0.2532981530343008
Precision:0.2910656920415019
Recall:0.2532981530343008
f1 score:0.25766107336542987


Applied k fold cross validation to confirm performance metrics

In [334]:
k_fold = KFold(n_splits = 5, random_state = None) 
score = cross_val_score(mlp, X, y, cv = k_fold)  
  
# Calculating mean accuracy score  
mean_acc_score = sum(score) / len(score)  
   
print("Accuracy score of each fold: ", score)  
print("Mean accuracy score: ", mean_acc_score)  

C:\Users\dhair\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\dhair\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy score of each fold:  [0.17941953 0.23280423 0.26719577 0.23015873 0.1957672 ]
Mean accuracy score:  0.22106909019837778


In [335]:
X.columns

Index(['season', 'city', 'team1', 'team2', 'toss_winner', 'toss_decision',
       'venue', 'umpire1', 'day_of_year'],
      dtype='object')

In [336]:
umpire_list = ['HDPK Dharmasena', 'M Erasmus', 'CB Gaffaney', 'MA Gough', 'AT Holdstock', 'RK Illingworth', 'RA Kettleborough', 'Nitin Menon', 'Ahsan Raza', 'PR Reiffel', 'Sharfuddoula', 'RJ Tucker', 'AG Wharf', 'JS Wilson', 'CM Brown', 'P Wilson']

In [337]:
sem_fin1_data = [[mapping_season['2023/24'],mapping_city['Mumbai'],TeamMapping['India'],TeamMapping['New Zealand'],'','',mapping_venue['Wankhede Stadium'],'',319]]
sem_fin2_data = [[mapping_season['2023/24'],mapping_city['Kolkata'],TeamMapping['South Africa'],TeamMapping['Australia'],'','',mapping_venue['Eden Gardens'],'',320]]
fin_data = [[mapping_season['2023/24'],mapping_city['Ahmedabad'],'','','','',mapping_venue['Narendra Modi Stadium'],'',323]]

sem_fin1_df = pd.DataFrame(sem_fin1_data,columns = X.columns)
sem_fin2_df = pd.DataFrame(sem_fin2_data,columns = X.columns)
fin_df =  pd.DataFrame(fin_data,columns = X.columns)
score = pd.DataFrame([[0,0,0,0]],columns = ['India','New Zealand','South Africa','Australia'])


In [338]:
sem_fin1_data

[[42, 89, 1, 7, '', '', 156, '', 319]]

In [339]:
sem_fin2_data

[[42, 77, 3, 2, '', '', 41, '', 320]]

### Creating dataframe for semi final match , for columns whose values are not available, filling in random values and doing this mutiple times and calculating result

### First winners of semi final matches are predicted and the results are plugged in dataframe of final match, prediction on this dataframe is the result, this result is calculated for multiple different situations and the team predicted majority of times as winner, is declared final winner 

### predicting world cup 2023 final winner using random forest model

In [340]:
import random
for i in range(100):
    
    #semi final 1
    if random.randrange(2):
        sem_fin1_df['toss_winner'] = sem_fin1_df['team1']
    else:
        sem_fin1_df['toss_winner'] = sem_fin1_df['team2']
        
    sem_fin1_df['toss_decision'] = random.randrange(2)
    sem_fin1_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    #semi final 2
    if random.randrange(2):
        sem_fin2_df['toss_winner'] = sem_fin2_df['team1']
    else:
        sem_fin2_df['toss_winner'] = sem_fin2_df['team2']
        
    sem_fin2_df['toss_decision'] = random.randrange(2)
    sem_fin2_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    sem_fin_pred1 = forest.predict(sem_fin1_df)
    sem_fin_pred2 = forest.predict(sem_fin2_df)
    
    #finals
    fin_df['team1'] =  sem_fin_pred1
    fin_df['team2'] =  sem_fin_pred2
    
    if random.randrange(2):
        fin_df['toss_winner'] = fin_df['team1']
    else:
        fin_df['toss_winner'] = fin_df['team2']
        
    fin_df['toss_decision'] = random.randrange(2)
    fin_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    #finals result
    fin_pred = forest.predict(fin_df)
    
    if fin_pred == 1 : score['India']+=1
    elif fin_pred == 3  : score['South Africa']+=1
    elif fin_pred == 7  : score['New Zealand']+=1
    else : score['Australia']+=1    
    
print("Chances of winning out of 100, according to random forest model")   
print(score)

Chances of winning out of 100, according to random forest model
   India  New Zealand  South Africa  Australia
0     91            3             6          0


In [341]:
print("Chances of winning out of 100, according to random forest model")   
print(score)

score = dict.fromkeys(score, 0)

Chances of winning out of 100, according to random forest model
   India  New Zealand  South Africa  Australia
0     91            3             6          0


### predicting world cup 2023 final winner using neural network model

In [342]:
import random
for i in range(100):
    
    #semi final 1
    if random.randrange(2):
        sem_fin1_df['toss_winner'] = sem_fin1_df['team1']
    else:
        sem_fin1_df['toss_winner'] = sem_fin1_df['team2']
        
    sem_fin1_df['toss_decision'] = random.randrange(2)
    sem_fin1_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    #semi final 2
    if random.randrange(2):
        sem_fin2_df['toss_winner'] = sem_fin2_df['team1']
    else:
        sem_fin2_df['toss_winner'] = sem_fin2_df['team2']
        
    sem_fin2_df['toss_decision'] = random.randrange(2)
    sem_fin2_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    sem_fin_pred1 = mlp.predict(sem_fin1_df)
    sem_fin_pred2 = mlp.predict(sem_fin2_df)
    
    #finals
    fin_df['team1'] =  sem_fin_pred1
    fin_df['team2'] =  sem_fin_pred2
    
    if random.randrange(2):
        fin_df['toss_winner'] = fin_df['team1']
    else:
        fin_df['toss_winner'] = fin_df['team2']
        
    fin_df['toss_decision'] = random.randrange(2)
    fin_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    #finals result
    fin_pred = mlp.predict(fin_df)
    
    if fin_pred == 1 : score['India']+=1
    elif fin_pred == 3  : score['South Africa']+=1
    elif fin_pred == 7  : score['New Zealand']+=1
    else : score['Australia']+=1    
    
print("Chances of winning out of 100, according to neural network model")   
print(score)

Chances of winning out of 100, according to neural network model
{'India': 82, 'New Zealand': 0, 'South Africa': 0, 'Australia': 18}


In [343]:
print("Chances of winning out of 100, according to neural network model")   
print(score)

score = dict.fromkeys(score, 0)

Chances of winning out of 100, according to neural network model
{'India': 82, 'New Zealand': 0, 'South Africa': 0, 'Australia': 18}


## Task1: 
### Predicting player of match

In [344]:
new_df.head()

,id,season,city,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,day_of_year
0,1389389,42,64,1,2,2,1,1,SS Iyer,56,50,267
1,1336129,41,96,5,12,12,1,5,WG Jacks,149,31,266
2,1395701,41,40,7,9,7,0,7,IS Sodhi,133,62,266
3,1389388,42,27,2,1,1,1,1,Mohammed Shami,100,57,265
5,1388414,41,34,4,1,4,0,1,Mohammed Siraj,106,10,260


## Handling null values

In [345]:
new_df.isna().sum()

id                  0
season              0
city                0
team1               0
team2               0
toss_winner         0
toss_decision       0
winner              0
player_of_match    18
venue               0
umpire1             0
day_of_year         0
dtype: int64

Dropping rows where player of match data not available

In [346]:
new_df = new_df.dropna(axis = 0,subset=['player_of_match'])
copy_df = copy_df.dropna(axis = 0,subset=['player_of_match'])

id column that is used for identification of row will not be helpful for prediction hence removing it

In [347]:
new_df = new_df.drop(['id'], axis=1)

In [348]:
new_df['player_of_match'] = le.fit_transform(new_df['player_of_match'])
# Create a dictionary mapping original values to encoded values
mapping_player = dict(zip(copy_df['player_of_match'], new_df['player_of_match']))

In [349]:
len(mapping_player)

493

## Data transformation

Splitting dataframe into target column and other columns,
and then further splitting into train and test

In [350]:
X = new_df[['season', 'city', 'team1', 'team2', 'toss_winner', 'toss_decision','winner',
        'venue', 'umpire1', 'day_of_year']]
y = new_df['player_of_match']


Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, random_state=42)

converting pandas dataframe to numpy as keras only takes numpy input

In [351]:
np_Xtrain = Xtrain.to_numpy(dtype='float32')
np_ytrain = ytrain.to_numpy(dtype='float32')
np_Xtest = Xtest.to_numpy(dtype='float32')
np_ytest = ytest.to_numpy(dtype='float32')

In [352]:
np_Xtrain, np_ytrain

(array([[ 33.,  31.,   3., ..., 113.,  92., 179.],
        [ 14.,  85.,   2., ...,  83.,  17.,  38.],
        [ 19.,  59.,   4., ...,  77.,  19., 206.],
        ...,
        [ 21.,  20.,  10., ..., 104., 110., 213.],
        [  9.,  86.,   9., ..., 133.,  36., 130.],
        [ 16.,  56.,   1., ...,  90.,  73., 332.]], dtype=float32),
 array([ 92., 414., 327., ..., 286., 293., 468.], dtype=float32))

Performing **One hot encoding** on target data , as it is required for classification

In [353]:
np_y = np.array([])
np_y = np.append(np_ytrain,np_ytest,axis=0)
np_y = to_categorical(np_y)


In [354]:
np_ytrain = np_y[:1498,]
np_ytest = np_y[1498:,]

In [355]:
np_ytrain.shape, np_ytest.shape

((1498, 493), (375, 493))

Applying Normalization on data

In [356]:
mean = np.mean(np_Xtrain, axis=0)
np_Xtrain -= mean
np_Xtest -= mean
std = np.std(np_Xtrain, axis=0)
np_Xtrain /= std
np_Xtest /= std

In [357]:
np_Xtrain.shape[-1], np_ytrain.shape[-1]

(10, 493)

Initializing neural network,
initializing number of hidden layers and number of neurons in each layer

In [358]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            128, activation="relu", input_shape=(np_Xtrain.shape[-1],)
        ),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(493, activation="softmax"),
    ]
)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               1408      
                                                                 
 dense_13 (Dense)            (None, 128)               16512     
                                                                 
 dense_14 (Dense)            (None, 128)               16512     
                                                                 
 dense_15 (Dense)            (None, 493)               63597     
                                                                 
Total params: 98,029
Trainable params: 98,029
Non-trainable params: 0
_________________________________________________________________


Using Adamprop optimizer and categorical cross entropy loss

In [359]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="categorical_crossentropy", metrics=metrics
)

In [360]:
np_Xtrain.shape,np_ytrain.shape,np_Xtest.shape, np_ytest.shape

((1498, 10), (1498, 493), (375, 10), (375, 493))

Training the model

In [361]:
model.fit(
    np_Xtrain,
    np_ytrain,
    epochs=80,
    verbose=2,
    validation_data=(np_Xtest, np_ytest),
)

Epoch 1/80
47/47 - 2s - loss: 5.9405 - fn: 1498.0000 - fp: 0.0000e+00 - tn: 737016.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 5.7166 - val_fn: 375.0000 - val_fp: 0.0000e+00 - val_tn: 184500.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 2s/epoch - 51ms/step
Epoch 2/80
47/47 - 0s - loss: 5.1236 - fn: 1498.0000 - fp: 0.0000e+00 - tn: 737016.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 5.6017 - val_fn: 375.0000 - val_fp: 0.0000e+00 - val_tn: 184500.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 272ms/epoch - 6ms/step
Epoch 3/80
47/47 - 0s - loss: 4.5765 - fn: 1498.0000 - fp: 0.0000e+00 - tn: 737016.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 5.4442 - val_fn: 375.0000 - val_fp: 0.0000e+00 - val_tn: 184500.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 277ms/epoch - 6ms/step
Epoch 4/80
4

Epoch 28/80
47/47 - 0s - loss: 0.9565 - fn: 560.0000 - fp: 256.0000 - tn: 736760.0000 - tp: 938.0000 - precision: 0.7856 - recall: 0.6262 - val_loss: 16.0289 - val_fn: 359.0000 - val_fp: 222.0000 - val_tn: 184278.0000 - val_tp: 16.0000 - val_precision: 0.0672 - val_recall: 0.0427 - 272ms/epoch - 6ms/step
Epoch 29/80
47/47 - 0s - loss: 0.8859 - fn: 531.0000 - fp: 251.0000 - tn: 736765.0000 - tp: 967.0000 - precision: 0.7939 - recall: 0.6455 - val_loss: 16.6489 - val_fn: 360.0000 - val_fp: 244.0000 - val_tn: 184256.0000 - val_tp: 15.0000 - val_precision: 0.0579 - val_recall: 0.0400 - 284ms/epoch - 6ms/step
Epoch 30/80
47/47 - 0s - loss: 0.8772 - fn: 519.0000 - fp: 240.0000 - tn: 736776.0000 - tp: 979.0000 - precision: 0.8031 - recall: 0.6535 - val_loss: 17.3336 - val_fn: 361.0000 - val_fp: 254.0000 - val_tn: 184246.0000 - val_tp: 14.0000 - val_precision: 0.0522 - val_recall: 0.0373 - 277ms/epoch - 6ms/step
Epoch 31/80
47/47 - 0s - loss: 0.8429 - fn: 501.0000 - fp: 251.0000 - tn: 736765.0

Epoch 55/80
47/47 - 0s - loss: 0.4388 - fn: 238.0000 - fp: 156.0000 - tn: 736860.0000 - tp: 1260.0000 - precision: 0.8898 - recall: 0.8411 - val_loss: 22.8934 - val_fn: 353.0000 - val_fp: 298.0000 - val_tn: 184202.0000 - val_tp: 22.0000 - val_precision: 0.0688 - val_recall: 0.0587 - 279ms/epoch - 6ms/step
Epoch 56/80
47/47 - 0s - loss: 0.4260 - fn: 225.0000 - fp: 164.0000 - tn: 736852.0000 - tp: 1273.0000 - precision: 0.8859 - recall: 0.8498 - val_loss: 24.8133 - val_fn: 358.0000 - val_fp: 302.0000 - val_tn: 184198.0000 - val_tp: 17.0000 - val_precision: 0.0533 - val_recall: 0.0453 - 296ms/epoch - 6ms/step
Epoch 57/80
47/47 - 0s - loss: 0.4358 - fn: 238.0000 - fp: 158.0000 - tn: 736858.0000 - tp: 1260.0000 - precision: 0.8886 - recall: 0.8411 - val_loss: 24.1278 - val_fn: 353.0000 - val_fp: 303.0000 - val_tn: 184197.0000 - val_tp: 22.0000 - val_precision: 0.0677 - val_recall: 0.0587 - 324ms/epoch - 7ms/step
Epoch 58/80
47/47 - 0s - loss: 0.4932 - fn: 255.0000 - fp: 162.0000 - tn: 73685

Model performance

In [362]:
model.evaluate(np_Xtest,np_ytest)

12/12 [==============================] - 0s 3ms/step - loss: 27.1202 - fn: 357.0000 - fp: 313.0000 - tn: 184187.0000 - tp: 18.0000 - precision: 0.0544 - recall: 0.0480


[27.120159149169922,
 357.0,
 313.0,
 184187.0,
 18.0,
 0.054380666464567184,
 0.04800000041723251]